In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
create or replace table dim_compute (
  compute_key string comment 'The primary key (PK) of the table.',
  workspace_id string comment 'The ID of the workspace where the warehouse is deployed.',
  account_id string comment 'The ID of the Databricks account.',
  warehouse_name string comment 'The name of the SQL warehouse.',
  warehouse_type string comment 'The type of SQL warehouse. Possible values are CLASSIC, PRO, and SERVERLESS.',
  warehouse_channel string comment 'The channel of the SQL warehouse. Possible values are CURRENT and PREVIEW.',
  warehouse_size string comment 'The cluster size of the SQL warehouse. Possible values are 2X_SMALL, X_SMALL, SMALL, MEDIUM, LARGE, X_LARGE, 2X_LARGE, 3X_LARGE, and 4X_LARGE.',
  min_clusters int comment 'The minimum number of clusters permitted.',
  max_clusters int comment 'The maximum number of clusters permitted.',
  auto_stop_minutes int comment 'The number of minutes before the SQL warehouse auto-stops due to inactivity.',
  change_time timestamp comment 'Timestamp of change to the SQL warehouse definition.',
  delete_time timestamp comment 'Timestamp of when the SQL warehouse was deleted. The value is null if the SQL warehouse is not deleted.',
  compute_type string not null comment 'The type of compute (Warehouse, Serverless, etc.)',
  primary key (compute_key) rely
)
comment 'DBSQL warehouse dimension sourced from system.compute.warehouses'
cluster by (compute_key);

In [0]:
insert overwrite dim_compute
-- Warehouses current row
select
  warehouse_id as compute_key,
  workspace_id,
  account_id,
  warehouse_name,
  warehouse_type,
  warehouse_channel,
  warehouse_size,
  min_clusters,
  max_clusters,
  auto_stop_minutes,
  change_time,
  delete_time,
  'Warehouse' as compute_type,
  row_number() over (partition by warehouse_id order by change_time desc) as rn
from system.compute.warehouses
qualify rn = 1
|> drop rn

union all

-- Serverless
select
  'serverless' as compute_key,
  'UNKNOWN' as workspace_id,
  'UNKNOWN' as account_id,
  'UNKNOWN' as warehouse_name,
  'UNKNOWN' as warehouse_type,
  'UNKNOWN' as warehouse_channel,
  'UNKNOWN' as warehouse_size,
  null as min_clusters,
  null as max_clusters,
  null as auto_stop_minutes,
  null as change_time,
  null as delete_time,
  'Serverless' as compute_type

union all

-- Unknown
select
  '-1' as compute_key,
  'UNKNOWN' as workspace_id,
  'UNKNOWN' as account_id,
  'UNKNOWN' as warehouse_name,
  'UNKNOWN' as warehouse_type,
  'UNKNOWN' as warehouse_channel,
  'UNKNOWN' as warehouse_size,
  null as min_clusters,
  null as max_clusters,
  null as auto_stop_minutes,
  null as change_time,
  null as delete_time,
  'UNKNOWN' as compute_type

In [0]:
optimize dim_compute

In [0]:
analyze table dim_compute compute statistics for all columns;

In [0]:
vacuum dim_compute